#**Lab 07-6-IMDB-introduction**

* ### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(777)
print(tf.__version__)

2.4.0


##**사용할 Data**

* Tensorflow 2.0 버전에 맞게 Keras를 활용한 IMDB 분류 모델 생성

##**학습에 쓰이는 Data**

* 50,000 movie reviews from the Internet Movie Database.

   (10000개의 빈도수가 높은 단얼르 학습시 Vector에 사용)

In [2]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))
print(train_data[0])

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


Training entries: 25000, labels: 25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


* IMDB Data를 Vector를 실제 값으로 변환하여 출력

In [3]:
#A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

#The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2 #unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

decode_review(train_data[4])

"<START> worst mistake of my life br br i picked this movie up at target for 5 because i figured hey it's sandler i can get some cheap laughs i was wrong completely wrong mid way through the film all three of my friends were asleep and i was still suffering worst plot worst script worst movie i have ever seen i wanted to hit my head up against a wall for an hour then i'd stop and you know why because it felt damn good upon bashing my head in i stuck that damn movie in the <UNK> and watched it burn and that felt better than anything else i've ever done it took american psycho army of darkness and kill bill just to get over that crap i hate you sandler for actually going through with this and ruining a whole day of my life"

In [4]:
print(train_labels[4])

0


##**Tensorflow Keras**

**위 Data를 기준으로 분류 모델을 생성**
* 학습과 평가를 위해 동일 길이인 256 길이의 단어로 PAD 값을 주어 맞춰줌

    (뒤의 길이는 0 값으로 맞춰줌)

In [5]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

print(len(train_data[0]), len(test_data[0]))
print(train_data[0])

256 256
[   1   14   22   16   43  530  973 1622 1385   65  458 4468   66 3941
    4  173   36  256    5   25  100   43  838  112   50  670    2    9
   35  480  284    5  150    4  172  112  167    2  336  385   39    4
  172 4536 1111   17  546   38   13  447    4  192   50   16    6  147
 2025   19   14   22    4 1920 4613  469    4   22   71   87   12   16
   43  530   38   76   15   13 1247    4   22   17  515   17   12   16
  626   18    2    5   62  386   12    8  316    8  106    5    4 2223
 5244   16  480   66 3785   33    4  130   12   16   38  619    5   25
  124   51   36  135   48   25 1415   33    6   22   12  215   28   77
   52    5   14  407   16   82    2    8    4  107  117 5952   15  256
    4    2    7 3766    5  723   36   71   43  530  476   26  400  317
   46    7    4    2 1029   13  104   88    4  381   15  297   98   32
 2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  10

* Tensorflow keras API를 통해 모델에 대한 정의
* 입력 Size와 학습시킬 Layer의 크기와 Activation Function 정의


In [6]:
#input shape is the vocabulary count used for the movie reviews(10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


* Adam Optimizer와 Cross Entropy Loss 선언

In [7]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

* 모델을 평가할 Test 데이터에 대한 정의 (10000을 기준으로 학습과 평가 수행)

In [8]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history  = model.fit(partial_x_train,
                     partial_y_train,
                     epochs=40,
                     batch_size=512,
                     validation_data=(x_val, y_val),
                     verbose=1)

Epoch 1/40
30/30 [==============================] - 2s 33ms/step - loss: 0.6928 - accuracy: 0.5008 - val_loss: 0.6906 - val_accuracy: 0.6589
Epoch 2/40
30/30 [==============================] - 1s 20ms/step - loss: 0.6889 - accuracy: 0.7048 - val_loss: 0.6835 - val_accuracy: 0.7414
Epoch 3/40
30/30 [==============================] - 1s 20ms/step - loss: 0.6795 - accuracy: 0.7546 - val_loss: 0.6693 - val_accuracy: 0.7444
Epoch 4/40
30/30 [==============================] - 1s 20ms/step - loss: 0.6618 - accuracy: 0.7722 - val_loss: 0.6461 - val_accuracy: 0.7646
Epoch 5/40
30/30 [==============================] - 1s 20ms/step - loss: 0.6329 - accuracy: 0.7828 - val_loss: 0.6132 - val_accuracy: 0.7778
Epoch 6/40
30/30 [==============================] - 1s 22ms/step - loss: 0.5962 - accuracy: 0.7970 - val_loss: 0.5741 - val_accuracy: 0.7979
Epoch 7/40
30/30 [==============================] - 1s 27ms/step - loss: 0.5495 - accuracy: 0.8162 - val_loss: 0.5312 - val_accuracy: 0.8146
Epoch 8/40
30

In [9]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 1s 2ms/step - loss: 0.3261 - accuracy: 0.8730
[0.3261283040046692, 0.8729599714279175]
